# Caso Práctico Unidad 3: Interpretabilidad

En este caso práctico vamos a trabajar con las técnicas de interpretabilidad de modelos vistas en la lección 6. Para ello en primer lugar necesitamos instalarnos las librerías necesarias. Las celdas siguientes, una vez ejecutadas e instaladas las librerías, se pueden borrar.

In [5]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=b89091877ee97dab349e02d555d7cab1fd70c265c1b65f9a87177188bbb82889
  Stored in directory: /Users/luismontalvo/Library/Caches/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [9]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.3/455.3 kB 7.9 MB/s eta 0:00:00a 0:00:01


In [ ]:
#Instala esta librería si no la instalanste en lecciones anteriores
#!pip install sklearn

## Librerías

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

#Módulos de interpretabilidad
import lime
import lime.lime_tabular
import shap

#Módulos de modelos
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#Módulos de evaluación de modelos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import f1_score, precision_score, recall_score


## Caso 1: Boston Housing

El conjunto de datos para este proyecto proviene del Repositorio de UCI Machine Learning Repository. Los datos de vivienda de Boston se recopilaron en 1978 y cada una de las 506 entradas representa datos agregados sobre 14 características de viviendas de varios suburbios de Boston, Massachusetts.

La variable objetivo es MEDV, precio promedio de la vivienda, en miles de dólares.

In [3]:
boston = pd.read_csv('data/BostonHousing.csv')
boston

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [15]:
features = boston.columns[:-1]
target = 'medv'

Podemos ver una descripción de los datos

In [5]:
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per $10,000
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in $1000's

## EDA

Realiza un anaísis de los datos para comprenderlos antes de modelar. Haz un anaálisis tanto univariante como multivariante.

### Análisis Univariante

Extrae medidas estadísticas, y plotea las variables para poder analizarlas.

¿Qué conclusiones extraes del análisis univariante de las variables?

### Análisis Multivariante

Estudia la correlación entre variables.

## Modelado

La variable objetivo es una variable continua, por lo que nos encontramos en un problema de aprendizaje supervisado de regresión.

Según lo visto en la Unidad 2, utiliza el algoritmo de Random Forest para predecir la variable objetivo. Una vez entrenado el modelo utiliza métricas de evaluación para analizar el rendimiento del mismo. En el caso en el que estas no sean óptimas, modifica hiperparámetros del algoritmo para mejorar la predicción.

### Random Forest

### Interpretabilidad Global

El algoritmo de random forest tiene incorporada la técnica de la importancia de variables lo cual nos puede dar información acerca de la interpretabildiad del modelo de forma global. Explora en la web de documentación de scikit-learn cómo poder extraer esta información del modelo.

### Interpretabilidad local: LIME

A continuación utilizaremos la librería `Lime`para hacer un análisis de las predicciones a nivel local. Al ser la primera vez que utilizamos esta librería el código se da a continuación.

In [ ]:
#Inicializamos el objeto explainer para regresión
explainer = lime.lime_tabular.LimeTabularExplainer(train[features].values, 
                                                    feature_names=features, 
                                                    class_names=target, 
                                                    categorical_features=features,
                                                    verbose=True, 
                                                    mode='regression')

Interpretamos la predición de algunas instancias específicas

In [ ]:
i = 50 #Elegimos una de las instancias que queremos analizar
print(i)
print("Real value:", test[target].iloc[i]) #Valor real de la instancia
exp = explainer.explain_instance(test[features].values[i], rf.predict, num_features=13) 
exp.show_in_notebook(show_table=True)

Cambia la instancia y analiza como cambia la predicción, y las variables en las que se basa para hacerla.

## Caso 2: Titanic

A continuación vamos a analizar otro caso con el dataset de Titanic que contiene información sobre los pasajeros del barco. La variable objetivo en este caso es la variable *survived* que indica con un 1 aquellos pasajeros que si sobrevivieron y con un 0 los que no.

### Modelo 1: Regresión Logística

La variable objetivo en este caso es una variable binaria, por lo que nos encontramos en un problema de aprendizaje supervisado de clasificación.

Selecciona únicamente las variables numéricas y predice con un algoritmo muy sencillo: la regresión logística.
Recuerda evaluar el rendimiento de tu modelo.

##### Interpretabilidad local: LIME

Tal y como se hizo en el caso 1, utiliza la librería `Lime`para analizar la predicción de instancias concretas. Por eje,plo la instancia número 1. Se proporciona el código de esta sección ya que es la primera vez que trabajarás con esta librería en clasificación.

In [ ]:
#Inicializamos el objeto explainer para clasificación
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=train_x.values,
                                                   mode="classification",
                                                   training_labels = train_y.values,
                                                   feature_names=train_x.columns.values.tolist())

In [ ]:
#Debemos obtener la probabilidad predicha por le modelo
lr_predict = lambda x: lr.predict_proba(x)

In [ ]:
instance_idx = 1
instance = test_x.iloc[instance_idx]
instance_true = test_y.iloc[instance_idx]

exp = explainer.explain_instance(instance,
                                 lr_predict)
exp.show_in_notebook(show_all=False)

print("ID: ",instance.name)
print("Name: ",df.loc[instance.name]["name"])
print("Predicting: ",int(exp.predict_proba[0]<exp.predict_proba[1]))
print("Did survive: ",instance_true)

La explicación consta de tres partes:

* La sección más a la izquierda muestra las probabilidades de predicción.
* La sección central devuelve las variables más importantes. Para esta tarea de clasificación binaria, estaría en 2 colores naranja/azul. Los atributos en naranja admiten la clase 1 y los de azul admiten la clase 0. Los números decimales en las barras horizontales representan la importancia relativa de estas características.
* La codificación de colores es consistente en todas las secciones. Contiene los valores reales de las 5 variables principales.

### Modelo 2: Árbol de decisión

Entra un modelo que admita todas las variables y analicemos sus predicciones. Vamos a trabajar con modelos basados en árboles.

#### EDA

A continuación realiza un análisis exploratorio de datos. Modifica las variables que sean necesarias según lo aprendido en la asignatura de EDA.

1. Imputa los valores nulos para poder ejecutar el modelo con todas las variables.

2. Elimina columnas no necesarias

3. Puedes realizar un tratamiento de los datos más detallado para practicar el análisis exploratorio de datos. Mapea a categorías las variables categóricas y cambia los tipos de columnas según creas necesarios. 

*Nota*: Algunas variables numéricas decimales tienen comas, python no reconoce la coma como separados decimal. Deberás cambiarlo por puntos para poder transformarlas a formato `float`

4. Estudia los valores de cada variable y analiza si todos los valores tienen sentido, en caso negativ, elimina estos registros.
5. Crea variables dummies para las variables categóricas que consideres.

#### Modelado

1. Entrenamiento
2. Predicción
3. Evlauación

##### Interpretabilidad local: LIME 

Analiza localmente la predicción de la instancia 2.

##### Importancia de variables

A continuación vamos a plotea el diagrama que  muestra las variables ordenadas según la importancia que han tenido para el modelo para poder tomar decisiones en la prediccion de la variable objetivo. El código se proporciona a continuación.

In [ ]:
feature_imp =  pd.Series(dt.feature_importances_, index = X_test.columns).sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(x=feature_imp, y = feature_imp.index, color = 'b')
plt.xlabel("Festure importance score")
plt.ylabel("Festures")
plt.title("Visualizing Important Features")
plt.tight_layout()

##### Decision tree plot

También se puede graficar el árbol de decisión y ver cómo ha formado las hojas y los splits para llegar a cada predicción. En él aparece para cada hoja el índice de Gini que vimos en la Unidad 1. El código se proporciona a continuación.

In [ ]:

fn = X_train.columns
cn = train_y.name
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (4,4), dpi=800)
tree.plot_tree(decision_tree=dt, feature_names = fn, class_names = cn, filled = True)
plt.show()

#### SHAP explainer

Vamos a usar la librería *shap* visto en teoría para interpretar los resultados del modelo en detalle, primero de una forma local para instancias concretas y después de forma general.


Los **SHAP values** para cada variable predictora representa el **cambio** en las predicciones esperadas por el modelo. Para cada una de estas variables, SHAP value explica la contribución para explicar la diferencia entre la media de la predicción del modelo y el valor real de la misma para cada instancia.

El código se proporciona a continuación.

In [ ]:

explainer = shap.TreeExplainer(dt)


In [ ]:
#Escogemos una instancia de nuestro conjunto de test
X_test.loc[[33]]

A continuación, generamos el gráfico de fuerza, que nos permite visualizar la predicción de la instancia superior a nivel local. Las variables mostradas a la izquierda han contribuido positivamente a la predicción (en este caso la clase en la que viajaba el pasajeto y el sexo del mismo) y la tarifa y el bote han contribuido negativamente.

In [ ]:
choosen_instance = X_test.loc[[33]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

Prueba a seleccionar otra instancia y analízala.

Podemos visualizar también un gráfico de importancia de variables, pero con la diferencia de que además nos da información de como dicha variable ha contribuido a la predicción de cada clase. En este caso vemos que las dos más importantes, sexo y la clase, contribuyen por igual tanto para la clase 1 como la 0.

In [ ]:
shap.summary_plot(shap_values, X_test)

#Si para la versión que tienes instalada de shap no funciona el summary_plot, puedes probar con el siguiente código
#shap.summary_plot(shap_values, X_test, plot_type="bar")

A continuación vamos a analizar el modelo de forma general. Para ellos graficaeremos las variables en orden descendiente de importancia, pero añadiendo un extra de información respecto a cuando las graficamos con el módulo correspondiente al árbol de decisión. Cada punto representa uuna instancia de los datos. El eje X representa el SHAP value, que como hemos dicho, es el impacto en la salida del modelo. 

Analiza el gráfico y obten conclusiones de la predicción del modelo.

In [ ]:
explainer = shap.Explainer(dt, X_test)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values[:,:,1])